In [1]:
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import pandas as pd

In [2]:
path='/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Drilling/_Shift Reports/'


In [3]:
def get_nan_inds(series):
    ''' Obtain the first and last index of each consecutive NaN group.
    '''
    series = series.reset_index(drop=True)
    index = series[series.notna()].index.to_numpy()
    if len(index) == 0:
        return []
    indices = np.split(index, np.where(np.diff(index) > 1)[0] + 1)
     
    groups=[(ind[0], ind[-1]+1) for ind in indices]
    return groups

In [4]:
name_list=[]
data_list=[]
for folder in os.listdir(path):
    level2=path+folder+'/'
    print(level2)
    if os.path.isdir(level2):
        for i,file in enumerate(os.listdir(level2)):
            print(i,file)
            file_path=level2+file

            try:
                data=pd.read_excel(file_path,na_filter=True)
                #data['FileName']=file
                data_list.append(data)
                name_list.append(file_path)
            except Exception as e:
                print(e)



/Volumes/GoogleDrive/Shared drives/AMC Projects/_AZ_Kay/_Drilling/_Shift Reports/BK-77/
0 Kay Mine 3_2.21.BK-77.xlsx
1 Kay Mine 3_2.20.BK-77.xlsx
2 Kay Mine 3_2.2.BK-77.xlsx
3 Kay Mine 3_2.17.BK-77.xlsx
4 Kay Mine 3_2.13.BK-77.xlsx
5 Kay Mine 3_2.19.BK-77.xlsx
6 Kay Mine 3_2.15.BK-77.xlsx
7 Kay Mine 3_2.14.BK-77.xlsx
8 Kay Mine 3_2.16.BK-77.xlsx
9 Kay Mine 3_2.18.BK-77.xlsx
10 Kay Mine 3_1.31.BK-77.xlsx
11 Kay Mine 3_2.1.BK-77.xlsx
12 Kay Mine 3_1.30.BK-77.xlsx
13 Kay Mine 3_1.28.BK-77.xlsx
14 Kay Mine 3_1.26.BK-77.xlsx
15 Kay Mine 3_1.29.BK-77.xlsx
16 Kay Mine 3_1.27.BK-77.xlsx
17 Kay Mine 3_2.12.BK-77.xlsx
18 Kay Mine 3_2.11.BK-77.xlsx
19 Kay Mine 3_2.10.BK-77.xlsx
20 Kay Mine 3_1.25.BK-77.xlsx
21 Kay Mine 3_1.24.BK-77.xlsx
22 Kay Mine 3_1.23.BK-77.xlsx
23 Kay Mine 3_1.20.BK-77.xlsx
24 Kay Mine 3_1.21.BK-77.xlsx
25 Kay Mine 3_1.19.BK-77.xlsx
26 Kay Mine 3_1.17.BK-77.xlsx
27 Kay Mine 3_1.18.BK-77.xlsx
28 Kay Mine 3_1.16.BK-77.xlsx
29 Kay Mine 3_1.22.BK-77.xlsx
30 Kay Mine 2_7.04.BK-67

In [5]:
def get_drill_data(data,indicators,comp_target=''):
    data=data=data.T.reset_index(drop=True).T

    data_columns=[]
    ind_head_groups=[]
    for col in data.columns:
        blocks=get_nan_inds(data[col])
        #print(blocks)
        block_list=[]
        for rows in blocks:
            start=rows[0]
            end=rows[-1]
            if start==end:
                end+=1
            data_col=data.loc[list(range(start,end)),col]
            block_list.append(data_col)
            for thing in indicators:
                if thing in data_col.to_list():
                    size=rows[1]-rows[0]
                    print((thing,rows,size))
                    ind_head_groups.append((thing,size))

        try:
            non_nan=pd.concat(block_list,axis=0)
            data_columns.append(non_nan)
        except Exception as e:
            print(e)
    data=pd.concat(data_columns,axis=1,ignore_index=True).reset_index(drop=True)
    data=data.astype('object')

    headers=[]
    for i,thing in enumerate(indicators):
        header=data.where(data.eq(thing)).stack().index.values[0][0]
        if thing in ind_head_groups[i]:
            size=ind_head_groups[i][1]
            print(thing,'header:',header,'size:',size)
        headers.append((header,size))
    headers
    slices=[]
    for i,header in enumerate(headers):
        head=header[0]
        size=header[1]
        columns = data.iloc[head].str.lower()
        
        try:
            start=head+1
            end=start+size
            
            print(head,'-',end)
            slice = pd.DataFrame(data.values[start:end], columns=columns).reset_index(drop=True)
        except Exception as e: print('Exception:',e)
        slices.append(slice)
    clean_data=pd.concat(slices,axis=1)

    clean_data=clean_data.dropna(how='all',axis=1).T.dropna(how='all',axis=0).T

    clean_data=clean_data.T.groupby(level=0).first().T
    if comp_target==''or comp_target=='water':
        for col in clean_data.columns:
            try:clean_data[col]=clean_data[col].str.replace(' ','')
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].str.replace('**','',regex=False)
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].str.replace('!','',regex=False)
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].str.strip('ft')
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].str.strip('gal')
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].replace(r'\s',' ',regex=True)
            except Exception as e: pass
            try:clean_data[col]=pd.to_numeric(clean_data[col])
            except Exception as e:pass
        try:    clean_data.columns=[col.replace('\n','') for col in clean_data.columns]
        except Exception as e: print(e)

        clean_data['drill from']=pd.to_numeric(clean_data['drill from'],errors='coerce')
        drop=clean_data[clean_data['drill from'].isna()==True].index
        clean_data=clean_data.drop(drop)
        drop_col=['name','shift','size','tools']
        for col in drop_col:
            try:clean_data=clean_data.drop(col,axis=1).fillna(0)
            except Exception as e: print(e)
        clean_data['total man hours']=pd.to_numeric(clean_data['total man hours'],errors='coerce').fillna(method='ffill').fillna(method='bfill')
        dr=data.where(data.eq('Drill')).stack().index.values[0]
        clean_data['drill']=data.iloc[dr[0],dr[1]+1]
        try:
            clean_data.rename(columns={'volume':'water_gal'},inplace=True)
        except Exception as e: print(e)

    if comp_target=='activities':
        clean_cols=['hole no.']
        for col in clean_cols:
            try:    clean_data[col]=clean_data[col].str.replace('**','',regex=False)
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].str.replace('!','',regex=False)
            except Exception as e: pass
            try:    clean_data[col]=clean_data[col].replace(r'\s',' ',regex=True)
            except Exception as e: pass
            try:    clean_data[col]=pd.to_numeric(clean_data[col])
            except Exception as e:pass
        try:    clean_data.columns=[col.replace('\n','') for col in clean_data.columns]
        except Exception as e: print(e)
        drop_col=['description','unit','quantity']
        for col in drop_col:
            try:clean_data=clean_data.drop(col,axis=1).fillna(0)
            except Exception as e: print(e)
        try: 
            hole=data.where(data.eq('Hole(s)')).stack().index.values[0]
            hole=data.iloc[hole[0],hole[1]+1]
            print(hole)
            clean_data['hole no.']=hole
        except Exception as e: print(e)
        drop=clean_data[clean_data['activity']==False].index
        clean_data=clean_data.drop(drop)
    date=data.where(data.eq('Date')).stack().index.values[0]
    clean_data['date']=data.iloc[date[0],date[1]+1]

    return clean_data

In [6]:
indicators=['Description']
comp_target='activities'
data=data_list[9]
data=get_drill_data(data,indicators=indicators,comp_target=comp_target)
data

('Description', (29, 39), 10)
No objects to concatenate
Description header: 13 size: 10
13 - 24
KM-22-63


,activity,hours,shift,time,type,hole no.,date
0,Hole Conditioning,2.5,Day,06:00-08:30,Hole Conditioning,KM-22-63,Feb-18-2022
1,Trip Rods,4.5,Day,08:30-13:00,Rod Handling,KM-22-63,Feb-18-2022
2,Drilling,2.5,Day,13:00-15:30,Core HQ Diamond Core Bit from 2487 ft to 2501 ...,KM-22-63,Feb-18-2022
3,Repair,2.5,Day,15:30-18:00,Rig Breakdown Royal fixed leak on head,KM-22-63,Feb-18-2022
4,Drilling,3.0,Night,18:00-21:00,Core HQ Diamond Core Bit from 2501 ft to 2522 ...,KM-22-63,Feb-18-2022
5,Hole Conditioning,5.5,Night,21:00-02:30,Hole Conditioning reamed from 900-bottom,KM-22-63,Feb-18-2022
6,Trip Rods,3.5,Night,02:30-06:00,Rod Handling,KM-22-63,Feb-18-2022


In [7]:
water_indicators=['Total Man Hours','Hole No.']
act_indicators=['Description']
water_data=[]
act_data=[]
for data in data_list:
    w_data=get_drill_data(data=data,indicators=water_indicators,comp_target='water')
    a_data=get_drill_data(data=data,indicators=act_indicators,comp_target='activities')

    water_data.append(w_data)
    act_data.append(a_data)
final_water=pd.concat(water_data,axis=0)
final_act=pd.concat(act_data,axis=0)

final_water=final_water.sort_values(['hole no.','date','drill from']).reset_index(drop=True)
final_cols=['hole no.','date','drill from','drill to','water_gal']
others=[col for col in final_water.columns if col not in final_cols]
final_water=pd.concat([final_water[final_cols],final_water[others]],axis=1)
final_act=final_act.sort_values(['hole no.','date']).reset_index(drop=True)
final_act.to_excel(path+'activity_compilation.xlsx',index=False)
final_water.to_excel(path+'water_compilation.xlsx',index=False)

('Total Man Hours', (14, 16), 2)
('Hole No.', (17, 20), 3)
No objects to concatenate
Total Man Hours header: 6 size: 2
Hole No. header: 9 size: 3
6 - 9
9 - 13
"['name'] not found in axis"
('Description', (29, 38), 9)
No objects to concatenate
Description header: 12 size: 9
12 - 22
KM-22-63
('Total Man Hours', (14, 16), 2)
('Hole No.', (17, 21), 4)
No objects to concatenate
Total Man Hours header: 6 size: 2
Hole No. header: 9 size: 4
6 - 9
9 - 14
('Description', (29, 38), 9)
No objects to concatenate
Description header: 13 size: 9
13 - 23
KM-22-63
('Total Man Hours', (14, 16), 2)
('Hole No.', (17, 21), 4)
No objects to concatenate
Total Man Hours header: 7 size: 2
Hole No. header: 10 size: 4
7 - 10
10 - 15
"['name'] not found in axis"
('Description', (29, 40), 11)
No objects to concatenate
Description header: 14 size: 11
14 - 26
KM-22-59A
('Total Man Hours', (14, 16), 2)
('Hole No.', (17, 21), 4)
No objects to concatenate
Total Man Hours header: 7 size: 2
Hole No. header: 10 size: 4
7 -

In [8]:
final_act

,activity,hours,shift,time,type,hole no.,date
0,Drilling,1.5,Day,06:00-07:30,Core PQ Diamond Core Bit from 0 ft to 12 ft.,KM - 22- 76,May-11-2022
1,Moving,1.5,Day,07:30-09:00,Align | Hole Alignment TN-14 rig aligner.,KM - 22- 76,May-11-2022
2,Moving,9.0,Day,09:00-18:00,Move | Rig Up/Down Cleared area with backhoe a...,KM - 22- 76,May-11-2022
3,Drilling,4.5,Night,18:00-22:30,Core PQ Diamond Core Bit from 12 ft to 70 ft. ...,KM - 22- 76,May-11-2022
4,Hole Conditioning,1.0,Night,22:30-23:30,Hole Conditioning,KM - 22- 76,May-11-2022
...,...,...,...,...,...,...,...
3291,Hole Conditioning,3.5,Day,14:30-18:00,Hole Conditioning sucking out sump.,"km_22_67, KM-22-66",Mar-02-2022
3292,Drilling,4.5,Night,18:00-22:30,Core PQ Diamond Core Bit from 0 ft to 58 ft wi...,"km_22_67, KM-22-66",Mar-02-2022
3293,Hole Conditioning,1.0,Night,22:30-23:30,Hole Conditioning,"km_22_67, KM-22-66",Mar-02-2022
3294,Moving,4.0,Night,23:30-03:30,Move | Rig Up/Down rigging down/up and moving ...,"km_22_67, KM-22-66",Mar-02-2022
